In [1]:
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pickle

import ray
import logging
from fastapi import FastAPI, Request
from ray import serve

In [2]:
# Load the data
dataset = np.loadtxt('data/pima-indians-diabetes.data.csv', delimiter=",")
# split data into X and y
X = dataset[:, 0:8]
y = dataset[:, 8]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=7)

In [3]:
def create_and_save_model():
    # Instantiate a model, fit and train
    xgb_model = XGBClassifier(use_label_encoder=False)
    xgb_model.fit(X_train, y_train)

    # saving the model
    with open('xgb_model.pkl', 'wb') as f:
        pickle.dump(xgb_model, f)

    return xgb_model

In [4]:
model = create_and_save_model()
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

/opt/homebrew/Caskroom/miniforge/base/envs/ray-core-tutorial/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


Accuracy: 74.02%


In [5]:
app = FastAPI()
if ray.is_initialized:
    ray.shutdown()
ray.init(namespace="xgbregressor", logging_level=logging.ERROR)

Python version:,3.8.13
Ray version:,2.2.0
Dashboard:,http://127.0.0.1:8265


In [6]:
@serve.deployment(num_replicas=2, route_prefix="/regressor")
@serve.ingress(app)
class XGBModel:
    def __init__(self):
        # loading the model
        with open("xgb_model.pkl", "rb") as f:
            self.model = pickle.load(f)
        print("Pickled XGBoost model loaded")

    @app.post("/")
    async def predict(self, starlette_request: Request):
        payload = await starlette_request.json()
        print("Worker: received starlette request with data", payload)

        prediction = round(self.model.predict([np.array(list(payload.values()))])[0])
        
        return {"result": prediction}

In [7]:
xgboost_handle_cls = XGBModel.bind()
serve.run(xgboost_handle_cls)

(ServeController pid=51457) INFO 2023-02-17 16:56:07,391 controller 51457 http_state.py:129 - Starting HTTP proxy with name 'SERVE_CONTROLLER_ACTOR:SERVE_PROXY_ACTOR-4ee775f2e68d29bb596d310c4fd3953166fb702c37db64e8e05f1ff5' on node '4ee775f2e68d29bb596d310c4fd3953166fb702c37db64e8e05f1ff5' listening on '127.0.0.1:8000'
(ServeController pid=51457) INFO 2023-02-17 16:56:08,062 controller 51457 deployment_state.py:1310 - Adding 2 replicas to deployment 'XGBModel'.
(HTTPProxyActor pid=51459) INFO:     Started server process [51459]


(ServeReplica:XGBModel pid=51462) Pickled XGBoost model loaded
(ServeReplica:XGBModel pid=51461) Pickled XGBoost model loaded


RayServeSyncHandle(deployment='XGBModel')

In [8]:
import requests

sample_request_inputs = [
    {"Pregnancies": 6,
     "Glucose": 148,
     "BloodPressure": 72,
     "SkinThickness": 35,
     "Insulin": 0,
     "BMI": 33.6,
     "DiabetesPedigree": 0.625,
     "Age": 50,
    },
    {"Pregnancies": 10,
      "Glucose": 168,
      "BloodPressure": 74,
      "SkinThickness": 0,
      "Insulin": 0,
      "BMI": 38.0,
      "DiabetesPedigree": 0.537,
      "Age": 34,
    },
    {"Pregnancies": 10,
     "Glucose": 39,
     "BloodPressure": 80,
     "SkinThickness": 0,
     "Insulin": 0,
     "BMI": 27.1,
     "DiabetesPedigree": 1.441,
     "Age": 57,
     },
     {"Pregnancies": 1,
      "Glucose": 103,
      "BloodPressure": 30,
      "SkinThickness": 38,
      "Insulin": 83,
      "BMI": 43.3,
      "DiabetesPedigree": 0.183,
      "Age": 33,
     }
    ]

In [9]:
# Iterate our requests
for sri in sample_request_inputs:
    response = requests.post("http://localhost:8000/regressor/", json=sri)
    print(response.text)

{"result":1}
{"result":1}
{"result":0}
{"result":0}
